In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "5"

In [2]:
import random

files = {
    "eng": "chat_timer_data_eng.txt",
    "rus": "chat_timer_data_rus.txt",
    "de": "chat_timer_data_de.txt",
}

train_lines = []
val_lines = []

for lang, file in files.items():
    with open(file, "r", encoding="utf-8") as f:
        all_lines = [line.strip() for line in f]

    unique_lines = list(set(all_lines))

    if lang == "eng" and len(unique_lines) > 1000:
        unique_lines = random.sample(unique_lines, 1000)

    random.shuffle(unique_lines)

    n_val = int(len(unique_lines) * 0.2)
    val = unique_lines[:n_val]
    train = unique_lines[n_val:]

    train_lines.extend(train)
    val_lines.extend(val)

with open("train.jsonl", "w", encoding="utf-8") as f:
    for line in train_lines:
        f.write(line + "\n")

with open("val.jsonl", "w", encoding="utf-8") as f:
    for line in val_lines:
        f.write(line + "\n")

print(f"Train lines: {len(train_lines)}")
print(f"Val lines: {len(val_lines)}")


Train lines: 2316
Val lines: 578


In [2]:
from datasets import load_dataset

dataset = load_dataset("json", data_files={"train": "train.jsonl", "validation": "val.jsonl"})

In [3]:
dataset["train"][0]

{'USER': 'Hey assistant, set a timer for 9 minutes.',
 'HOURS': 0,
 'MINUTES': 9,
 'SECONDS': 0}

In [4]:
def format_prompt(sample):
    user = sample["USER"]
    hours = sample["HOURS"]
    minutes = sample["MINUTES"]
    seconds = sample["SECONDS"]
    return f"USER:{user}\nHOURS:{hours}\nMINUTES:{minutes}\nSECONDS:{seconds}"

In [5]:
dataset["train"] = dataset["train"].map(lambda x: {"text": format_prompt(x)})
dataset["validation"] = dataset["validation"].map(lambda x: {"text": format_prompt(x)})

In [6]:
dataset["train"][0]

{'USER': 'Hey assistant, set a timer for 9 minutes.',
 'HOURS': 0,
 'MINUTES': 9,
 'SECONDS': 0,
 'text': 'USER:Hey assistant, set a timer for 9 minutes.\nHOURS:0\nMINUTES:9\nSECONDS:0'}

In [7]:
print(dataset["train"][0]["text"])

USER:Hey assistant, set a timer for 9 minutes.
HOURS:0
MINUTES:9
SECONDS:0


In [8]:
from transformers import AutoTokenizer

model_name = "google/gemma-3-270m"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [9]:
max([len(tokenizer(sample["text"])["input_ids"]) for sample in dataset["train"]])

44

In [10]:
max([len(tokenizer(sample["text"])["input_ids"]) for sample in dataset["validation"]])

37

In [11]:
tokenizer.padding_side

'left'

In [12]:
def tokenize(sample):
    prompt = f"USER:{sample["USER"]}\n"
    prompt_len = len(tokenizer(prompt)["input_ids"])
    tokenized = tokenizer(sample["text"], padding="max_length", max_length=45)
    pad_len = tokenized["input_ids"].count(tokenizer.pad_token_id)
    tokenized["labels"] = tokenized["input_ids"].copy()
    tokenized["labels"][: pad_len + prompt_len] = [-100] * (pad_len + prompt_len)
    return tokenized

dataset["train"] = dataset["train"].map(tokenize, batched=False)
dataset["validation"] = dataset["validation"].map(tokenize, batched=False)

In [13]:
dataset["train"][0].keys()

dict_keys(['USER', 'HOURS', 'MINUTES', 'SECONDS', 'text', 'input_ids', 'attention_mask', 'labels'])

In [14]:
print(dataset["train"][0]["attention_mask"])
print(dataset["train"][0]["input_ids"])
print(dataset["train"][0]["labels"])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 20791, 236787, 17531, 16326, 236764, 1076, 496, 20342, 573, 236743, 236819, 4310, 236761, 107, 10858, 66481, 236787, 236771, 107, 16008, 80914, 236787, 236819, 107, 149542, 236787, 236771]
[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 10858, 66481, 236787, 236771, 107, 16008, 80914, 236787, 236819, 107, 149542, 236787, 236771]


In [15]:
from transformers import (
    AutoModelForCausalLM,
    TrainingArguments,
)
from peft import LoraConfig
from trl import SFTTrainer


model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    use_cache=False,
    attn_implementation="eager",
)


In [16]:
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
)

In [17]:
training_args = TrainingArguments(
    output_dir="./gemma-timer-lora-multilang",
    num_train_epochs=20,
    per_device_train_batch_size=64,
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    logging_steps=10,
    eval_strategy="steps",
    eval_steps=25,
    save_strategy="steps",
    save_steps=25,
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    report_to=[],
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    peft_config=peft_config,
    args=training_args,
)

trainer.train()


WARN  Python GIL is enabled: Multi-gpu quant acceleration for MoE models is sub-optimal and multi-core accelerated cpu packing is also disabled. We recommend Python >= 3.13.3t with Pytorch > 2.8 for mult-gpu quantization and multi-cpu packing with env `PYTHON_GIL=0`.
WARN  Feature `utils/Perplexity` requires python GIL or Python >= 3.13.3T (T for Threading-Free edition of Python) plus Torch 2.8. Feature is currently skipped/disabled.
INFO  ENV: Auto setting PYTORCH_CUDA_ALLOC_CONF='expandable_segments:True' for memory saving.
INFO  ENV: Auto setting CUDA_DEVICE_ORDER=PCI_BUS_ID for correctness.          


INFO:root:cc -pthread -fno-strict-overflow -Wsign-compare -Wunreachable-code -DNDEBUG -g -O3 -Wall -fPIC -fPIC -c /tmp/tmpxvw1pqg9/test.c -o /tmp/tmpxvw1pqg9/test.o
INFO:root:cc -pthread /tmp/tmpxvw1pqg9/test.o -laio -o /tmp/tmpxvw1pqg9/a.out
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
INFO:root:cc -pthread -fno-strict-overflow -Wsign-compare -Wunreachable-code -DNDEBUG -g -O3 -Wall -fPIC -fPIC -c /tmp/tmp4rd8t5zd/test.c -o /tmp/tmp4rd8t5zd/test.o
INFO:root:cc -pthread /tmp/tmp4rd8t5zd/test.o -L/usr/local/cuda-12.6 -L/usr/local/cuda-12.6/lib64 -lcufile -o /tmp/tmp4rd8t5zd/a.out
INFO:root:cc -pthread -fno-strict-overflow -Wsign-compare -Wunreachable-code -DNDEBUG -g -O3 -Wall -fPIC -fPIC -c /tmp/tmpnjdzexie/test.c -o /tmp/tmpnjdzexie/test.o
INFO:root:cc -pthread /tmp/tmpnjdzexie/test.o -laio -o /tmp/tmpnjdzexie/a.out
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
The tokenizer 

Step,Training Loss,Validation Loss,Entropy,Num Tokens,Mean Token Accuracy
25,0.198700,0.086829,2.116249,138780.000000,0.976405
50,0.046300,0.045152,2.028481,277560.000000,0.988949
75,0.030700,0.030087,1.563409,416340.000000,0.992267
100,0.018000,0.024895,1.215853,549900.000000,0.993987
125,0.011200,0.022359,1.216305,688680.000000,0.996189
150,0.008000,0.020613,0.906997,827460.000000,0.996070
175,0.007900,0.021335,0.827084,961020.000000,0.996066
200,0.004100,0.018274,0.811551,1099800.000000,0.996447
225,0.004500,0.019273,0.754587,1238580.000000,0.996934
250,0.002900,0.016644,0.730340,1372140.000000,0.997181


KeyboardInterrupt: 

In [18]:
trainer.save_model("./gemma-timer-lora-multilang")

In [21]:
from itertools import product

sample_template = "HOURS:{}\nMINUTES:{}\nSECONDS:{}"

max_new_tokens = max(
    [
        len(tokenizer.tokenize(sample_template.format(h, m, s)))
        for h, m, s in product(range(100), repeat=3)
    ]
)

KeyboardInterrupt: 

In [24]:
max_new_tokens

16

In [19]:
max_new_tokens = 16

In [20]:
from transformers import logging

logging.set_verbosity_error()

In [21]:
from transformers import pipeline
from peft import PeftModel

base_model = AutoModelForCausalLM.from_pretrained(model_name, device_map="cuda")
model = PeftModel.from_pretrained(base_model, "./gemma-timer-lora-multilang", device_map="cuda")
text_gen = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [22]:
from tqdm import tqdm


def evaluate_accuracy(
    dataset,
    log_file,
    batch_size=64,
):
    correct = 0
    total = len(dataset)

    with open(log_file, "w") as file:
        for i in tqdm(range(0, total, batch_size)):
            texts = dataset[i : i + batch_size]["text"]
            prefixes = [text.split("\n")[0] + "\n" for text in texts]
            gen_outs = text_gen(prefixes, max_new_tokens=max_new_tokens, num_beams=1, do_sample=False)
            for text, gen_out in zip(texts, gen_outs):
                gen_text = gen_out[0]["generated_text"]
                if len(gen_text) >= len(text) and text == gen_text[: len(text)]:
                    correct += 1
                else:
                    print(f"Mismatch: {text} -> {gen_text}\n", file=file)

    return correct / total

In [23]:
acc = evaluate_accuracy(dataset["validation"], log_file="eval_multilang.log")
print(f"Validation accuracy: {acc:.4f}")

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [05:21<00:00, 32.19s/it]

Validation accuracy: 0.9291


In [25]:
%cat eval_multilang.log

Mismatch: USER:Set a timer for three quarters of an hour.
HOURS:0
MINUTES:45
SECONDS:0 -> USER:Set a timer for three quarters of an hour.
HOURS:30
MINUTES:15
SECONDS:0


Mismatch: USER:Set a timer to run for sixty eight minutes
HOURS:0
MINUTES:68
SECONDS:0 -> USER:Set a timer to run for sixty eight minutes
HOURS:0
MINUTES:0
SECONDS:68
SECONDS

Mismatch: USER:start a timer for fifty seven minutes
HOURS:0
MINUTES:57
SECONDS:0 -> USER:start a timer for fifty seven minutes
HOURS:0
MINUTES:0
SECONDS:57
SECONDS

Mismatch: USER:start a timer for a quarter of an hour
HOURS:0
MINUTES:15
SECONDS:0 -> USER:start a timer for a quarter of an hour
HOURS:30
MINUTES:30
SECONDS:0


Mismatch: USER:create a timer lasting 20 minutes
HOURS:0
MINUTES:20
SECONDS:0 -> USER:create a timer lasting 20 minutes
HOURS:0
MINUTES:0
SECONDS:20
SECONDS

Mismatch: USER:set a 7 minute and 7 second timer
HOURS:0
MINUTES:7
SECONDS:7 -> USER:set a 7 minute and 7 second timer
HOURS:7
MINUTES:7
SECONDS:7
SECONDS:

Mismatch: U